# Day 1

In [26]:
import pandas as pd
import numpy as np

In [62]:
df = pd.read_csv('inputs/day1.txt', header=None)
x = df.iloc[:,0]

In [65]:
df.head()

,0
0,193
1,195
2,204
3,208
4,219


In [67]:
df.shift(1).bfill().head()

,0
0,193.0
1,193.0
2,195.0
3,204.0
4,208.0


In [75]:
(df.head() > df.shift(1).bfill().head()).astype(int).sum()

0    4
dtype: int64

## Part one

In [76]:
((x - x.shift(1).bfill()).values.flatten() > 0).sum() 

1832

## Part two

In [4]:
wx = x.rolling(window=3).sum().dropna()
((wx - wx.shift(1).bfill()).values.flatten() > 0).sum() 

1858

# Day 2

In [13]:
df = pd.read_csv('inputs/day2.txt', header=None, delimiter=' ')
df.columns = ['direction', 'val']
df.direction.unique()

array(['forward', 'down', 'up'], dtype=object)

## Part one

In [17]:
forward = df[df.direction == 'forward'].val.sum()
down = df[df.direction == 'down'].val.sum()
up = df[df.direction == 'up'].val.sum()
forward * (down-up)

1936494

## Part two

In [19]:
hori, aim, depth = 0, 0, 0
for cmd, val in df.values:
    if cmd == 'down':
        aim += val
    if cmd == 'up':
        aim -= val
    if cmd == 'forward':
        hori += val
        depth += aim*val
hori*depth

1997106066

# Day 3

In [41]:
# Load
df = pd.read_csv('inputs/day3.txt', header=None, names=['raw'], dtype={'raw': str})
#n_bits = df.raw
n_bits = df.raw.str.len().unique()[0]
print(f'n_bits = {n_bits}')
df.head()

n_bits = 12


,raw
0,111100101100
1,101100110001
2,100110100101
3,001101100010
4,010111011110


## Part one

In [46]:
gamma_str = "".join([
    str(int(df.raw.str.slice(start=i, stop=i+1).astype(int).sum() > len(df) / 2))
    for i in range(n_bits)
])
gamma = int(gamma_str, 2)
epsilon_str = "".join(['1' if s == '0' else '0' for s in gamma_str])
epsilon = int(epsilon_str, 2)
print(f'{gamma_str} => {gamma}')
print(f'{epsilon_str} => {epsilon}')
print(f'result = {gamma} * {epsilon} = {gamma * epsilon}')

000011011010 => 218
111100100101 => 3877
result = 218 * 3877 = 845186


## Part two

In [54]:
oxy = df.copy()
i = 0
while len(oxy) > 1:
    most_common = str(int(oxy.raw.str.slice(start=i, stop=i+1).astype(int).sum() >= len(oxy) / 2))
    oxy = oxy[oxy.raw.str.slice(start=i, stop=i+1) == most_common]
    i = (i + 1) % n_bits
oxy_rating_str = oxy.raw.values[0]
oxy_rating = int(oxy_rating_str, 2)

co2 = df.copy()
i = 0
while len(co2) > 1:
    most_common = str(int(co2.raw.str.slice(start=i, stop=i+1).astype(int).sum() < len(co2) / 2))
    co2 = co2[co2.raw.str.slice(start=i, stop=i+1) == most_common]
    i = (i + 1) % n_bits
co2_rating_str = co2.raw.values[0]
co2_rating = int(co2_rating_str, 2)

print(f'{oxy_rating_str} => {oxy_rating}')
print(f'{co2_rating_str} => {co2_rating}')
print(f'life support rating = {oxy_rating} * {co2_rating} = {oxy_rating * co2_rating}')

010110110011 => 1459
110001101010 => 3178
life support rating = 1459 * 3178 = 4636702


# Day 4

In [38]:
# Load data
with open('input4.txt') as fi:
    lines = fi.readlines()
len(lines)

def get_boards(lines):
    rows = []
    for line in lines:
        line = line.strip()
        if not line: continue
        rows.append([int(s) for s in line.split()])
        if len(rows) == 5:
            yield np.array(rows)
            rows = []

all_draws = [int(s) for s in lines[0].strip().split(',')]
boards = list(get_boards(lines[1:]))

print(all_draws)
print(boards[0])
print(boards[0].T)

[26, 38, 2, 15, 36, 8, 12, 46, 88, 72, 32, 35, 64, 19, 5, 66, 20, 52, 74, 3, 59, 94, 45, 56, 0, 6, 67, 24, 97, 50, 92, 93, 84, 65, 71, 90, 96, 21, 87, 75, 58, 82, 14, 53, 95, 27, 49, 69, 16, 89, 37, 13, 1, 81, 60, 79, 51, 18, 48, 33, 42, 63, 39, 34, 62, 55, 47, 54, 23, 83, 77, 9, 70, 68, 85, 86, 91, 41, 4, 61, 78, 31, 22, 76, 40, 17, 30, 98, 44, 25, 80, 73, 11, 28, 7, 99, 29, 57, 43, 10]
[[57 12 60 96 93]
 [73 87 63 70 91]
 [74 32 43 67 46]
 [59 34  5 35 82]
 [53 40 55 29  1]]
[[57 73 74 59 53]
 [12 87 32 34 40]
 [60 63 43  5 55]
 [96 70 67 35 29]
 [93 91 46 82  1]]


## Part one

In [61]:
def check_full(row, draws):
    measure = set(row).difference(draws)
    return len(measure) == 0
    

def find_board(boards, all_draws):
    for round_i in range(len(all_draws)):
        draw = all_draws[round_i]
        draws = all_draws[:round_i+1]
        for board_i, board in enumerate(boards):
            for row in np.vstack([board, board.T]):
                if check_full(row, draws):
                    return draw, draws, board, board_i, row


draw, draws, board, board_i, row = find_board(boards, all_draws)
unmarked = list(set(board.flatten()).difference(draws))
print(f'Board = {board_i}')
print(board)
print(f'Draws = [{draw}], {draws}')
print(f'Unmarked = {unmarked}')


score = np.array(unmarked).sum() * draw
print(f'Score = {score}')

Board = 43
[[ 8 32 94 72 74]
 [27 29 22  2 76]
 [58 54 80  5 35]
 [36 24 83 59 25]
 [21 31 48 39  4]]
Draws = [94], [26, 38, 2, 15, 36, 8, 12, 46, 88, 72, 32, 35, 64, 19, 5, 66, 20, 52, 74, 3, 59, 94]
Unmarked = [4, 76, 80, 83, 21, 22, 24, 25, 27, 29, 31, 39, 48, 54, 58]
Score = 58374


## Part two

In [68]:
def check_full(row, draws):
    measure = set(row).difference(draws)
    return len(measure) == 0
    

def find_board(boards, all_draws):
    remain = np.ones(len(boards))
    for round_i in range(len(all_draws)):
        draw = all_draws[round_i]
        draws = all_draws[:round_i+1]
        for board_i, board in enumerate(boards):
            for row in np.vstack([board, board.T]):
                if check_full(row, draws):
                    remain[board_i] = 0
            if remain.sum() == 0:
                return draw, draws, board, board_i, row

draw, draws, board, board_i, row = find_board(boards, all_draws)
unmarked = list(set(board.flatten()).difference(draws))
print(f'Board = {board_i}')
print(board)
print(f'Winning row = {row}')
print(f'Draws = [{draw}], {draws}')
print(f'Unmarked = {unmarked}')


score = np.array(unmarked).sum() * draw
print(f'Score = {score}')

Board = 16
[[41 99 93 62 96]
 [90 30 10  5 94]
 [98 32 83 78 25]
 [76 27 29 19 35]
 [58 91 34 31  3]]
Winning row = [96 94 25 35  3]
Draws = [31], [26, 38, 2, 15, 36, 8, 12, 46, 88, 72, 32, 35, 64, 19, 5, 66, 20, 52, 74, 3, 59, 94, 45, 56, 0, 6, 67, 24, 97, 50, 92, 93, 84, 65, 71, 90, 96, 21, 87, 75, 58, 82, 14, 53, 95, 27, 49, 69, 16, 89, 37, 13, 1, 81, 60, 79, 51, 18, 48, 33, 42, 63, 39, 34, 62, 55, 47, 54, 23, 83, 77, 9, 70, 68, 85, 86, 91, 41, 4, 61, 78, 31]
Unmarked = [98, 99, 10, 76, 25, 29, 30]
Score = 11377
